#   <u>Satellite Cloud Coverage and Power Generation Visualization</u>


## INTRODUCTION

This notebook generates a time-lapse video showcasing the movement of clouds across the sky along with the corresponding power generation at each PV system on a daily basis. By combining the satellite-based cloud coverage data from the EUMETSAT Solar Forecasting dataset and the power generation information, the video illustrates the dynamic relationship between cloud movement and the amount of power generated at each PV system.

**Data**

there are three datasets used for this analysis:


- **eumetsat_seviri_hrv_uk.zarr**: The cloud coverage dataset is derived from the EUMETSAT SEVIRI RSS, which covers the northern third of the Meteosat disc and captures images every five minutes.

  The dataset used for this analsis,available on Google Cloud, was transformed by Open Climate Fix and is a subset of the   complete SEVIRI RSS dataset. It includes data for the United Kingdom and North Western Europe from January 2020 to November 2021. The original data is protected by copyright held by EUMETSAT. However, EUMETSAT has granted permission for the distribution of this transformed dataset. For more information please read here https://console.cloud.google.com/marketplace/product/bigquery-public-data/eumetsat-seviri-rss-hrv-uk?project=steel-apparatus-318910



- **5min parquet**: Time series data of PV solar generation data. Available: https://huggingface.co/datasets/openclimatefix/uk_pv/tree/main. For information about the data read more here https://huggingface.co/datasets/openclimatefix/uk_pv
- **metadata**: Metadata of the different PV systems. Available: https://huggingface.co/datasets/openclimatefix/uk_pv/tree/main. Read more here https://huggingface.co/datasets/openclimatefix/uk_pv

**Method**

- Step 1) Utilizing cloud coverage satellite images from the EUMETSAT Solar Forecasting dataset, extracting the cloud image foreground
- Step 2) Superimposing the cloud image it onto the power generation data of each PV system. for a specific time stamp
- Step 3) Iterate throught all time stamps in one day ( from 00:00:00 to 23:55:00) and export as HTML
- Step 4) take a screen shot of HTML files and export to png
- Step 5) Combine the PNG images into a video to create a time-lapse representation of the cloud movement and power generation.

## ANALYSIS

In [ ]:
# Installing requiered packages
!pip install xarray[complete]
!pip install ocf-blosc2
!pip install opencv-python
!pip install geopandas
!pip install pyshp
!pip install Pillow
!pip install scikit-image
!pip install selenium
!pip install imgkit
!pip install moviepy

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


from shapely.geometry import Point
from pyproj import Proj, transform
from pyproj import CRS, Transformer
from skimage import color, filters
import xarray as xr
import ocf_blosc2
import folium
import pyproj
import cv2

import time
import os
import datetime

import folium

### Loading datasets

#### REPLACE PATH

In [ ]:
# define paths to data
math_to_min5 = PATH # REPLACE with path to 5min data
math_to_metadata = PATH # REPLACE with path to metadata

In [ ]:
# Load the 5min data file and metadata
min5 = pd.read_csv(math_to_min5)
# Import metadata
meta = pd.read_csv(math_to_metadata)

### Prepairing Data

The data will be loaded and cleaned using the following four functions

1) The *load_data(time)* function loads the EUMETSAT Solar Forecasting dataset for a specific time.

2) The *select_transform(data_in, timestamp)* function selects a specific time slice from the input dataset, converts it to a pandas dataframe, and transforms the coordinate system from OSGB (British National Grid) to WGS84 (latitude and longitude).

3) The *pwg_gen_clean(loading_date, loading_time)* function cleans and filters power generation data for a specific date and time. It also merges the filtered data with additional metadata and creates a GeoDataFrame with a geometry column based on latitude and longitude information.

4) The *create_map(dataset)* function processes satellite data by creating a cloud mask and generating a transparent cloud image. It then creates a folium map centered around the UK and adds power generation data as circles on the map, with circle size and fill color based on the power generation value. The function overlays the transparent cloud image on the map and saves it as an HTML file.

In [ ]:
def load_data(time):
    'loads the EUMETSAT Solar Forecasting dataset for a specific time'
    try:
        df = (
            xr.open_mfdataset(
                "gs://public-datasets-eumetsat-solar-forecasting/satellite/EUMETSAT/SEVIRI_RSS/v3/eumetsat_seviri_hrv_uk.zarr",
                engine="zarr",
                concat_dim="time",
                combine="nested",
                chunks={},
                join="override",
            )
            .drop_duplicates("time")
            .sel(time=time)
        )
        return df
    except KeyError:
        return None

def select_transform(data_in, timestamp):
    #'select a specific time slice from the input dataset, convert the dataset to
    #'a pandas dataframe, transforms the coordinate system from OSGB'
    #'(British National Grid) to WGS84 (latitude and longitude).'

    # Define the input and output coordinate systems
    input_crs = CRS.from_epsg(27700)  # OSGB (British National Grid)
    output_crs = CRS.from_epsg(4326)  # WGS84 (latitude and longitude)

    # Select the specific time slice and extract the data, convert to pandas
    data_slice = data_in.to_dataframe()

    # Create a transformer to convert between coordinate systems
    transformer = Transformer.from_crs(input_crs, output_crs)

    # Convert x_osgb and y_osgb to latitude and longitude
    data_slice['latitude'], data_slice['longitude'] = transformer.transform(
        data_slice['x_osgb'].values.round(2),
        data_slice['y_osgb'].values.round(2)
    )
    return data_slice


def pwg_gen_clean(loading_date, loading_time):
    'cleans and filters power generation data for a specific date and time'
    'merges the filtered data with the metadata and creates a'
    'GeoDataFrame with a geometry column based on latitude and longitude'
    # Clean the string columns for date and time
    min5['date'] = min5['date'].str.strip()
    min5['time'] = min5['time'].str.strip()
    # Filter for the desired date, this will reduce the size of the dataset
    min5_sel = min5[min5['date'] == loading_date]
    # Fixing the timestamp
    min5_sel.loc[:, 'timestamp'] = pd.to_datetime(min5_sel['time'], format='%H:%M').dt.time
    # Add the new column to the dataframe
    min5_sel.loc[:, 'timestamp'] = min5_sel['date'].astype('str') + ' ' + min5_sel['timestamp'].astype('str')
    # Drop unnecessary columns
    min5_sel = min5_sel.drop(['time', 'date'], axis=1)
    # Convert timestamp to datetime
    min5_sel.loc[:, 'timestamp'] = pd.to_datetime(min5_sel['timestamp'])
    # Filter the dataset for the specified time
    time2 = time.replace('T', ' ')
    filtered_data = min5_sel.loc[min5_sel['timestamp'] == pd.Timestamp(time2)]
    # Merging with the 5min data
    columns_to_include = ['ss_id', 'latitude_rounded', 'longitude_rounded']
    filtered_data_sel = filtered_data.merge(meta[columns_to_include], on='ss_id', how='left')
    # Create a geometry column using the latitude and longitude information
    geometry = [Point(xy) for xy in zip(filtered_data_sel['longitude_rounded'], filtered_data_sel['latitude_rounded'])]
    # Convert the pandas DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(filtered_data_sel, geometry=geometry)
    return gdf


def create_map(dataset):
    #'processes satellite data by creating'
    #'a cloud mask and generating a transparent cloud image'
    #'creates a folium map centered around the UK and adds power generation data
    #'as circles on the map, with circle size and fill color based on the power generation value'
    #'Then overlays the transparent cloud image on the map and saves it as an HTML file.'

    # Process satellite data
    image = np.array(dataset['data'])
    image_gray = image.reshape((891, 1843))
    image_gray_normalized = image_gray / image_gray.max()
    threshold = 0.2
    cloud_mask = image_gray_normalized > threshold
    cloud_image = image_gray.copy()
    cloud_image[~cloud_mask] = 0
    transparent_cloud_image = np.zeros((cloud_image.shape[0], cloud_image.shape[1], 4), dtype=np.uint8)
    transparent_cloud_image[..., 2] = cloud_image
    transparent_cloud_image[..., 3] = cloud_mask.astype(np.uint8) * 255

    # Create a folium map centered around the UK
    m = folium.Map(location=[54.7023545, -3.2765753], zoom_start=6)

    # Get the latitude and longitude range
    min_lat, max_lat = dataset['latitude'].min(), dataset['latitude'].max()
    min_lon, max_lon = dataset['longitude'].min(), dataset['longitude'].max()

    # Add the power generation data as circles on the map
    pw_gen_gdf = pwg_gen_clean(loading_date, loading_time)
    for i in range(len(pw_gen_gdf)):
        power_generation = float(pw_gen_gdf.iloc[i]['generation_wh'])

        fill_color = 'grey'
        if power_generation < 50:
            fill_color = '#4DD0E1'
            radius=1000
        elif power_generation < 100:
            fill_color = '#0097A7'
            radius=5000
        else:
            fill_color = '#006064'
            radius=10000

        folium.Circle(
            location=[pw_gen_gdf.iloc[i]['latitude_rounded'], pw_gen_gdf.iloc[i]['longitude_rounded']],
            radius=radius,
            color=fill_color,
            fill=True,
            fill_color=fill_color
        ).add_to(m)

    # Overlay the transparent cloud image
    transparent_cloud_image[..., :3] = 255

    folium.raster_layers.ImageOverlay(
        image=transparent_cloud_image,
        bounds=[[min_lat, min_lon], [max_lat, max_lon]],
        opacity=0.7,
    ).add_to(m)

    # Save the map as an HTML file
    valid_time = time.replace(":", "_")
    save_name = f"map_{valid_time}.html"
    m.save(save_name)

### RUNNING THE FUNCTIONS

This code is performing the following tasks:

1) Sets an initial timestamp such as "2021-04-25T00:00:00". This value needs to be changed based on the target date

2) Enters a loop that iterates through 5-minute intervals until the end of a 24-hour cycle.

3) Within each iteration, it extracts the date and time components from the current timestamp.

4) Calls the load_data(time) function to load the data for the specified timestamp.

5) Checks if the timestamp is present in the loaded dataframe. If not, it moves to the next timestamp by incrementing the time by 5 minutes.

6) If the timestamp is found in the dataframe, it calls the pwg_gen_clean(loading_date, loading_time) function to process power generation data for the specified date and time.

7) Calls the select_transform(df, time) function to transform and select the satellite data for the specified time.

8) Calls the create_map(satalight) function to create a map visualization based on the transformed satellite data and power generation data.

9) After processing the current timestamp, it increments the time by 5 minutes and combines the date and time components to prepare for the next iteration.

10) The loop continues until completing the 24-hour cycle.

In [ ]:
import datetime

# Set the initial time
time = "2021-04-25T23:55:00"

# Iterate through 5-minute intervals until the end of the 24-hour cycle
while time <= "2021-04-25T23:55:00":
    # Extract the date component
    loading_date = time[:10]

    # Convert the date component to a datetime object
    loading_date_dt = datetime.datetime.strptime(loading_date, "%Y-%m-%d")

    # Format the date component in the desired format
    loading_date_formatted = loading_date_dt.strftime("%Y-%m-%d")

    # Extract the time component
    loading_time = time[11:]
    print("loading_date:", loading_date_formatted)
    print("loading_time:", loading_time)

    # Calling functions
    df = load_data(time)

    # Check if the timestamp is present in the dataframe
    if df is None:
        print("Timestamp not found in the dataframe. Moving to the next timestamp.")
        # Increment the time by 5 minutes
        loading_time_dt = datetime.datetime.strptime(loading_time, "%H:%M:%S")
        loading_time_dt += datetime.timedelta(minutes=5)
        loading_time = loading_time_dt.strftime("%H:%M:%S")

        # Combine the date and time components
        time = loading_date + "T" + loading_time
        continue

    pw_gen_gdf = pwg_gen_clean(loading_date, loading_time)
    satalight = select_transform(df, time).reset_index()
    create_map(satalight)

    # Increment the time by 5 minutes
    loading_time_dt = datetime.datetime.strptime(loading_time, "%H:%M:%S")
    loading_time_dt += datetime.timedelta(minutes=5)
    loading_time = loading_time_dt.strftime("%H:%M:%S")

    # Combine the date and time components
    time = loading_date + "T" + loading_time

*There are missing date included in the dataframe, some days are missing a substantian amount of data, for example June 2 2021


### CONVERT TO PNG & CREATE VIDEO

This code automates the process of capturing screenshots of web pages, adding custom text, cropping the images, and creating a video from the processed images.

For each HTML file:
1) Loads the HTML file in the browser
2) Captures a screenshot of the web page and saves it as a PNG file.
3) Opens the PNG image and crops the image to a specific region, adds text specifying the time, and saves the modified image.
4) Creates a video from the generated PNG images by writing each image as a frame in the video.

#### REPLACE PATH

In [ ]:
# Define paths to data
html_path = PATH # replace with input path (where the html files were saved from the
output_path = PATH #replace with output path
video_name = 'JAN01' # repalce with video name in the format JAN01 for example

# Configure Selenium web driver
driver = webdriver.Chrome()  # Replace with the appropriate driver for your browser

In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from selenium import webdriver
import cv2

In [ ]:
# Create the output folder
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get the list of HTML files in the folder
html_files = [f for f in os.listdir(html_path) if f.endswith('.html')]

for html_file in html_files:
    # Set the input and output file paths
    input_file = os.path.join(html_path, html_file)

    # Extract the time value from the file name
    time_str = os.path.splitext(html_file)[0].split("map_")[1]
    time = datetime.strptime(time_str, "%Y-%m-%dT%H_%M_%S").strftime("%H_%M")

    # Create the output file name with the time value
    output_file = os.path.join(output_path, f"{time}.png")

    # Load the HTML file in the browser
    driver.get(f"file://{input_file}")

    # Capture a screenshot of the web page
    driver.save_screenshot(output_file)

    # Open the generated PNG image
    image = Image.open(output_file)

    # Define the crop region coordinates (left, upper, right, lower)
    left = 52
    upper = 52
    right = min(image.width, 5000)
    lower = min(image.height, 5000)
    crop_region = (left, upper, right, lower)

    # Crop the image using the defined region
    image = image.crop(crop_region)

    # Add custom text to the image
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("courbd.ttf", 15)
    text = f"TIME:{time.replace('_', ':')}"
    draw.text((100, 100), text, fill=(0, 0, 0), font=font)

    # Save the modified image
    image.save(output_file)

# Close the browser and quit the driver
driver.quit()

# Create the video from the generated images
image_folder = output_path
video_name = os.path.join(output_path, f"{video_name}.avi")

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

fps = 24  # Desired frame rate

# Define the FourCC code for the codec (in this case, XVID)
fourcc = cv2.VideoWriter_fourcc(*'XVID')

video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

## TESTING

This section is provided to enable users to evaluate the performance of the cloud coverage mask. There may be situations where the cloud coverage image does not accurately display clouds, resulting in a white cast over the video. The purpose of this section is to verify that such issues are not caused by the processing steps, but rather reflect the original data. By conducting tests in this section, users can ensure the integrity of the generated images and assess the accuracy of the cloud representation.

In [ ]:
# Set the initial time
time = "2021-01-01T05:40:00"

# Extract the date component
loading_date = time[:10]

# Convert the date component to a datetime object
loading_date_dt = datetime.strptime(loading_date, "%Y-%m-%d")

# Format the date component in the desired format
loading_date_formatted = loading_date_dt.strftime("%Y-%m-%d")

# Extract the time component
loading_time = time[11:]
print("loading_date:", loading_date_formatted)
print("loading_time:", loading_time)

# Call the functions to prepair data
df = load_data(time)
pw_gen_gdf = pwg_gen_clean(loading_date, loading_time)
satalight = select_transform(df, time).reset_index()

# Load the immages
image = np.array(satalight['data'])
image_gray = image.reshape((891, 1843))

# Apply an appropriate threshold to segment the clouds
image_gray_normalized = image_gray / image_gray.max()
threshold = 0.2
cloud_mask = image_gray_normalized > threshold

# Extract the cloud region from the original image
cloud_image = image_gray.copy()
cloud_image[~cloud_mask] = 0

# Create a copy of the cloud image with transparent background
transparent_cloud_image = np.zeros((cloud_image.shape[0], cloud_image.shape[1], 4), dtype=np.uint8)
transparent_cloud_image[..., 2] = cloud_image
transparent_cloud_image[..., 3] = cloud_mask.astype(np.uint8) * 255

plt.imshow(cloud_image)

In [ ]:
plt.imshow(cloud_mask)